## Booking.com accomodation affiliate link
In this notebook the goal is to prototype and test a tool that will automatically detect mentions of accomodation in a blog post and then edit that mention to be an affiliate link to booking.com.


### Booking.com Affiliate Partner Program
To generate revenue from affiliate links in booking.com you must sign up for the Booking.com Affiliate Partner Program. Once accepted, you'll receive an affiliate id (`BOOKING_AID`) which must be part of the hyperlink in order for you to get paid for a booking made using your link.  
  
First we will need to extract mentions of accomodation in a text then we will construct the link to booking.com.  

The affiliate link should be in the form 
"https://www.booking.com/searchresults.html?ss=<`extracted_accomodation_name`>&aid=`BOOKING_AID`".



# Accomodation mention search

First step in the project is to extract mentions of accomodation in a text.  
There are a few methods we could use to do this such as using keyword and pattern matching, eg. finding Capitalised words or searching for sentences containing words such as "Hotel", "Apartment", "B&B" etc using libraries such as regex but it will miss ambigious matches eg. "The Four Seasons Singapore", it would be hard to maintain, you'll have to keep the keyword list updated.  

I decided to go with an LLM based method for the following reasons:  
- Flexibility - can deal with paraphrasing, slang and unclear contexts.
-  No training data or *regex* required. xD
- Will work across multiple languages, very important for an international travel blog where accomodation names may be in the local language on booking.com.
- Personal knowledge/education